# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [96]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


Paso 2: Descargar HTML

In [97]:
url="https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records"
headers={
   "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"
}

response=requests.get(url, time.sleep(10), headers=headers)

response.status_code

200

Paso 3: Transforma el HTML

In [98]:
if response:
    soup=BeautifulSoup(response.text, 'html')

tablas = soup.find_all("table", class_="wikitable sortable plainrowheaders")

In [103]:
filas = tablas[0].find_all('tr')
data = []

for fila in filas:
    celdas = fila.find_all(['th', 'td'])
    celdas_clean = []
    for c in celdas:
        celdas_clean.append(c.text.strip())
    data.append(celdas_clean)


Paso 4: Procesa el DataFrame

In [105]:
df = pd.DataFrame(data[1:], columns = data[0])
df = df.drop(100)
df["Streams(billions)"] = pd.to_numeric(df["Streams(billions)"])
df["Rank"] = pd.to_numeric(df["Rank"])
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rank               100 non-null    int64  
 1   Song               100 non-null    str    
 2   Artist(s)          100 non-null    str    
 3   Streams(billions)  100 non-null    float64
 4   Release date       100 non-null    str    
 5   Ref.               100 non-null    str    
dtypes: float64(1), int64(1), str(4)
memory usage: 4.8 KB


Paso 5: Almacena los datos en sqlite

In [ ]:
con = sqlite3.connect("webscraping.db")

df.to_sql('tabla', con, if_exists='replace', index=False)
con.commit()
#Probando que se creara nuestra tabla
test = pd.read_sql('SELECT * FROM tabla', con)
con.close()
test

,Rank,Song,Artist(s),Streams(billions),Release date,Ref.
0,1,"""Blinding Lights""",The Weeknd,5.273,29 November 2019,[1]
1,2,"""Shape of You""",Ed Sheeran,4.765,6 January 2017,[2]
2,3,"""Sweater Weather""",The Neighbourhood,4.375,3 December 2012,[3]
3,4,"""Starboy""",The Weeknd and Daft Punk,4.359,21 September 2016,[4]
4,5,"""As It Was""",Harry Styles,4.243,1 April 2022,[5]
...,...,...,...,...,...,...
95,96,"""Payphone""",Maroon 5 and Wiz Khalifa,2.547,16 April 2012,[96]
96,97,"""Dreams""",Fleetwood Mac,2.541,4 February 1977,[97]
97,98,"""Sicko Mode""",Travis Scott and Drake,2.531,21 August 2018,[98]
98,99,"""Someone Like You""",Adele,2.521,24 January 2011,[99]
